# How to transform shapes in 2D...
## ... with the little help of linear algebra!

In [ ]:
from numpy import array, identity as I
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from math import pi as π, cos, sin
from random import random

# A bit of randomness...
σ = lambda x = .5: x*(random() - 0.5)

# Transformation of the polygon's vertices
transform = lambda P, points: array([P @ p for p in points])
# Homogeneous to Euclidean coordinates conversion
h2e = lambda X: array([(x/x[-1])[:-1] for x in X])

### A square (a kinky one...) 

In [ ]:
# The original shape at the origin
sqwk = array([[0, 0, 1], [1, 0, 1], [1, 1, 1], [.25, .25, 1], [0, 1, 1]])

# Identity transformation (a.k.a. "do nothing but with great style!")
sqwk = transform(I(3), sqwk)
verticesI = h2e(sqwk)

## Translation/shift

In [ ]:
# Our square after translation by K and L...
K, L = 1.5 + σ(), 1 + σ(2)
T = I(3); _T = array([K, L])
T[:2, 2] = _T.T # Transposition seems superfluous here, but it's a good habit

## Rotation

In [ ]:
# ... and after rotation by α degrees...
α = σ(30)*π/180
R = I(3); _R = array([[cos(α), -sin(α)],
                      [sin(α),  cos(α)]])
R[:2, :2] = _R

## Scaling, squeezing, stretching & shearing – [Procrustes](https://en.wikipedia.org/wiki/Procrustes) would've loved it!

In [ ]:
# ... and scaling...
s = σ(2)
S = I(3); _S = array([[s, 0],
                      [0, s]])
S[:2, :2] = _S

## And...
#  See https://en.wikipedia.org/wiki/Transformation_matrix#/media/File:2D_affine_transformation_matrix.svg
#  and https://en.wikipedia.org/wiki/Transformation_matrix#/media/File:Perspective_transformation_matrix_2D.svg

# ... stretching:
k, κ = 2 + σ(), 2 + σ()
S1 = array([[k, 0, 0],
            [0, κ, 0],
            [0, 0, 1]])
# ... squeezing:
S2 = array([[1/κ, 0, 0],
            [0,   κ, 0],
            [0,   0, 1]])
# and shearing:
S3 = array([[1, k, 0],
            [κ, 1, 0],
            [0, 0, 1]])

## Reflections (ThnX, [Hauseholder](https://en.wikipedia.org/wiki/Alston_Scott_Householder)!)

In [ ]:

# Reflecting:
lx, ly = .5 + σ(), .5 + σ()   # ... about the line from origin through (lx, ly)
M = I(3)
_M = array([[lx**2 - ly**2,     2*lx*ly  ],
            [    2*lx*ly  , ly**2 - lx**2]])
_M = _M/(lx**2 + ly**2)
M[:2, :2] = _M

## All in a single step!

In [ ]:
# Feel free to further squeeze, stretch, shear the shape (and mix the order too)...
sqwk = transform(M@T@S@R@S3, sqwk)
verticesII = h2e(sqwk)


# Projections

In [ ]:
# Set the stage!
# https://www.geeksforgeeks.org/how-to-draw-shapes-in-matplotlib-with-python/
fig, ax = plt.subplots()
ax.set_xlim(-1, 4); ax.set_ylim(-1, 4); ax.set_aspect('equal'); plt.grid(True)

# Eventually, the final projection (a.k.a. casting a shadow)...
# https://youtu.be/27vT-NWuw0M, https://youtu.be/JK-8XNIoAkI and https://youtu.be/cTyNpXB92bQ
# which is a linear operation too and can be represented by a matrix as well:
# https://wrfranklin.org/pmwiki/Main/HomogeneousCoords
# The shadow of the final shape on the line Ax + By = 1
A, B = 1 + σ(), 1 + σ()
P = array([[1, 0, 0],
           [0, 1, 0],
           [A, B, 0]])
shdw = transform(P, sqwk); verticesIII = h2e(shdw)

# https://stackoverflow.com/questions/44526364/fill-matplotlib-polygon-with-a-gradient-between-vertices
for (vertices, edges, fills) in [(verticesI, 'green', 'lightgreen'),
                                 (verticesII, 'blue', 'lightblue'),
                                 (verticesIII, 'red', 'tomato')]:
    polygon = Polygon(vertices, ec = edges, fc = fills, alpha = 0.5)
    ax.scatter(vertices[:, 0], vertices[:, 1], color = edges, s = 25)
    ax.add_patch(polygon)


# And the ♪♫Moonlight shadows♫♪... https://youtu.be/ixExC-Zgyzc and https://youtu.be/e80qhyovOnA of the vertices.
# Is that song about Évariste Galois https://en.wikipedia.org/wiki/%C3%89variste_Galois#Final_days?
for v in sqwk:
    # Homogeneous to Euclidean coordinates
    x, y = v[:-1]/v[-1]
    ax.plot((0, x), (0, y), c = 'lightgray', lw = .5, ls = 'dashed')

plt.title('Matrix transforms in 2D: rigid, linear, affine & projective'); plt.show()